<a href="https://colab.research.google.com/github/pysjays777/project-j/blob/main/Project_J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe, get_as_dataframe

scopes = ['https://www.googleapis.com/auth/spreadsheets']
creds = Credentials.from_service_account_file(
    r"C:\Users\oem\Desktop\Learning journey\my.json",
    scopes=scopes
)
gc = gspread.authorize(creds)

sh1 = gc.open_by_key("1bDoPogywjpAu47VQmjJRzNcG3p4M7iScRKfAv8KokAI")
ws1 = sh1.worksheet("Sheet1")
df1 = get_as_dataframe(ws1)
print(df1)
df1 = df1[['symbol', 'Price', 'Change_%', 'Volume_USDT']]
df1['Source'] = 'Targerted Sheet 1'

sh2 = gc.open_by_key("1wfuz34LxJKEB15oEvxyIq7jE8lnltNqB7bFRJvshJMU")
ws2 = sh2.worksheet("Sheet1")
df2 = get_as_dataframe(ws2)
print(df2)
df2 = df2[['symbol', 'Price', 'Change_%', 'Volume_USDT']]
df2['Source'] = 'Targerted Sheet 2'

sh3 = gc.open_by_key("13es3JOoZtuvwoYdKqCSGwsfhaouAoQsw_8PJYkG74ZI")
ws3 = sh3.worksheet("Sheet1")
df3 = get_as_dataframe(ws3)
print(df3)
df3 = df3[['symbol', 'Price', 'Change_%', 'Volume_USDT']]
df3['Source'] = 'Targerted Sheet 3'

merged = pd.concat([df1, df2, df3], ignore_index=True)
merged['rowid'] = merged.index + 1

sh_dashboard = gc.open_by_key("15lD_Jp4xuFmacTMsFvcgOUpzrL5pdAO7uO4elMfe8rI")
try:
    ws_dashboard = sh_dashboard.worksheet("Dashboard")
except gspread.WorksheetNotFound:
    ws_dashboard = sh_dashboard.add_worksheet(title="Dashboard", rows="1000", cols="30")

set_with_dataframe(ws_dashboard, merged)
print(" Data written to Main Dashboard")

Import Data from Binance API

In [ ]:
from pathlib import Path
import requests
import pandas as pd
import os, pathlib
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

scopes = ['https://www.googleapis.com/auth/spreadsheets']
creds = Credentials.from_service_account_file(
    r"C:\Users\oem\Desktop\Learning journey\my.json",
    scopes=scopes
)

gc = gspread.authorize(creds)
sh = gc.open_by_key("1bDoPogywjpAu47VQmjJRzNcG3p4M7iScRKfAv8KokAI")
worksheet = sh.worksheet("Sheet1")



url = 'https://api.binance.com/api/v3/ticker/24hr'
response = requests.get(url)
data = response.json()
coins_i_want = ['BTCUSDT','ETHUSDT','ADAUSDT','PEPEUSDT','PIUSDT','XRPUSDT','SOLUSDT']
filtered_data = [ {key: item[key] for key in ['symbol','lastPrice', 'priceChangePercent','volume','quoteVolume']}
                 for item in data if item['symbol'] in coins_i_want]
df = pd.DataFrame(filtered_data)


df.rename(columns={
   'lastPrice': 'Price',
   'priceChangePercent': 'Change_%',
   'quoteVolume': 'Volume_USDT',
   'volume': 'Volume_coin',}, inplace=True)


cols = ['Volume_USDT','Volume_coin']
df[cols] = df[cols].astype(float).round(0).astype(int)


cols_to_format = ['Volume_USDT','Volume_coin']
df[cols_to_format] = df[cols_to_format].applymap(lambda x: '{:,}'.format(x))


print(df)
print(df.dtypes)

out_dir  = Path(r"C:\Users\oem\Desktop\Learning journey")
out_dir.mkdir(parents=True, exist_ok=True)

out_file = out_dir / "binancetest.csv"

df.to_csv(out_file, index=False)
print(" Data saved to:", out_file)

set_with_dataframe(worksheet, df)
print(" Data written to Google Sheets")